## Predict the Criminals

#### Installation of the XGBoost

In [3]:
!conda install -c conda-forge xgboost --yes

Solving environment: ...working... failed



PackagesNotFoundError: The following packages are not available from current channels:

  - xgboost

Current channels:

  - https://conda.anaconda.org/conda-forge/win-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/free/win-64
  - https://repo.anaconda.com/pkgs/free/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/pro/win-64
  - https://repo.anaconda.com/pkgs/pro/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




<img src="flow_chart.png" height=200px width=800px></img>

## 01 : Frame the Problem

The dataset contains the answers to private and personal life of people.Given this, we have to predict whether a person is criminal or not.The train data consists of 45718 rows, while the test data consists of 11430 rows.

#### Problem Link:https://www.hackerearth.com/challenge/competitive/predict-the-criminal/machine-learning/predict-the-criminal/

## 02 : Obtain Data

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier      #
from sklearn import cross_validation, metrics  #to include metrics for evaluation
from sklearn.grid_search import GridSearchCV #to use gridsearchcv
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt
%matplotlib inline

ModuleNotFoundError: No module named 'xgboost'

### Get the Data

In [4]:
crim_data=pd.read_csv('criminal_train.csv')

## 03 : Analyze Data 

In [5]:
crim_data.head()

,PERID,IFATHER,NRCH17_2,IRHHSIZ2,IIHHSIZ2,IRKI17_2,IIKI17_2,IRHH65_2,IIHH65_2,PRXRETRY,...,TOOLONG,TROUBUND,PDEN10,COUTYP2,MAIIN102,AIIND102,ANALWT_C,VESTR,VEREP,Criminal
0,25095143,4,2,4,1,3,1,1,1,99,...,1,2,1,1,2,2,3884.805998,40026,1,0
1,13005143,4,1,3,1,2,1,1,1,99,...,2,2,2,3,2,2,1627.108106,40015,2,1
2,67415143,4,1,2,1,2,1,1,1,99,...,2,2,2,3,2,2,4344.957980,40024,1,0
3,70925143,4,0,2,1,1,1,1,1,99,...,2,2,1,1,2,2,792.521931,40027,1,0
4,75235143,1,0,6,1,4,1,1,1,99,...,2,2,2,2,2,2,1518.118526,40001,2,0


In [6]:
crim_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45718 entries, 0 to 45717
Data columns (total 72 columns):
PERID        45718 non-null int64
IFATHER      45718 non-null int64
NRCH17_2     45718 non-null int64
IRHHSIZ2     45718 non-null int64
IIHHSIZ2     45718 non-null int64
IRKI17_2     45718 non-null int64
IIKI17_2     45718 non-null int64
IRHH65_2     45718 non-null int64
IIHH65_2     45718 non-null int64
PRXRETRY     45718 non-null int64
PRXYDATA     45718 non-null int64
MEDICARE     45718 non-null int64
CAIDCHIP     45718 non-null int64
CHAMPUS      45718 non-null int64
PRVHLTIN     45718 non-null int64
GRPHLTIN     45718 non-null int64
HLTINNOS     45718 non-null int64
HLCNOTYR     45718 non-null int64
HLCNOTMO     45718 non-null int64
HLCLAST      45718 non-null int64
HLLOSRSN     45718 non-null int64
HLNVCOST     45718 non-null int64
HLNVOFFR     45718 non-null int64
HLNVREF      45718 non-null int64
HLNVNEED     45718 non-null int64
HLNVSOR      45718 non-null int64
IRMCDCH

## Brief Introduction On the Model Being Used:

    Ensemble:
        It is a collection of predictors which are put together to give the final prediction.This is done beacause a collection of predictors together give better performance than a single predictor.There are two techniques of ensembling:
        Bagging:
            This involves building independent predictors and combining all the predictors to give the final prediction
        Boosting:
            This involves building the predictors sequentially rather than building each predictor independently.In these predictors learn from the mistakes commited by previous predictors.
    Gradient Boosting:
        Gradient boosting is a machine learning technique for regression and classification problems, which produces a prediction model in the form of an ensemble of weak prediction models, typically decision trees.One of the widely used algorithms in gradient boosting is XGBoost algorithm.
    XGBoost(eXtreme Gradient Boosting):
        XGBoost is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable. It implements machine learning algorithms under the Gradient Boosting framework. XGBoost provides a parallel tree boosting (also known as GBDT, GBM) that solve many data science problems in a fast and accurate way. 

#### XgBoostClassifier:
    It is an implementation of sklearn api for XgBoost Classification.The parameters of the XGBClassifier are:
        booster [default=gbtree]:
            Select the type of model to run at each iteration.
        silent [default=0]:
            Silent mode is activated is set to 1, i.e. no running messages will be printed.
        nthread [default to maximum number of threads available if not set]:
            This is used for parallel processing and number of cores in the system should be entered
        eta [default=0.3]:
            learning rate in GBM
        min_child_weight [default=1]:
            Defines the minimum sum of weights of all observations required in a child.
        max_depth [default=6]:
            The maximum depth of a tree.Used to control over-fitting as higher depth will allow model to learn relations 
            very specific to a particular sample.
        max_leaf_nodes:
            The maximum number of terminal nodes or leaves in a tree.Can be defined in place of max_depth.
        gamma [default=0]:
        A node is split only when the resulting split gives a positive reduction in the loss function.Gamma specifies 
        the minimum loss reduction required to make a split.
        subsample [default=1]:
            Denotes the fraction of observations to be randomly samples for each tree.
        colsample_bytree [default=1]:
            Denotes the fraction of columns to be randomly samples for each tree.
        colsample_bylevel [default=1]:
            Denotes the subsample ratio of columns for each split, in each level.
    

## 04 and 05 : Feature Engineering and Model Selection 

<img src="supervised_flow_chart.png"></img>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(crim_data.drop('Criminal',axis=1), 
                                                    crim_data['Criminal'], test_size=0.30, 
                                                    random_state=101)

In [ ]:
train=pd.concat([X_train,y_train],axis=1)

In [ ]:
#function to estimate the best value of n_estimators and fit the model with the given data

In [ ]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        #to get the parameters of xgboost
        xgb_param = alg.get_xgb_params() 
        
        #to convert into a datastructure internally used by xgboost for training efficiency 
        # and speed
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        
        #xgb.cv is used to find the number of estimators required for the parameters 
        # which are set
        cvresult = xgb.cv(xgb_param, xgtrain, 
                          num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
                        metrics='auc', early_stopping_rounds=early_stopping_rounds)
        
        #setting the n_estimators parameter using set_params
        alg.set_params(n_estimators=cvresult.shape[0])
        
        print(alg.get_xgb_params())
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Criminal'],eval_metric='auc')
    
    return alg

In [ ]:
#function to get the accuracy of the model on the test data given the features considered

In [ ]:
def get_accuracy(alg,predictors):
    dtrain_predictions = alg.predict(X_test[predictors])
    dtrain_predprob = alg.predict_proba(X_test[predictors])[:,1]
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(y_test.values, 
                                                      dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_test.values, 
                                                           dtrain_predprob))

In [ ]:
#function to get the feature importances based on the model fit

In [ ]:
def get_feature_importances(alg):
    #to get the feature importances based on xgboost we use fscore
    feat_imp = pd.Series(alg._Booster.get_fscore()).sort_values(ascending=False)
    print(feat_imp)
    
    #this shows the feature importances on a bar chart
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
target = 'Criminal'
IDcol = 'PERID'

In [ ]:
#to return the XGBClassifier object based on the values of the features

In [ ]:
def XgbClass(learning_rate =0.1,n_estimators=1000,max_depth=5,min_child_weight=1,
             gamma=0,subsample=0.8,colsample_bytree=0.8):
    xgb1 = XGBClassifier(learning_rate=learning_rate,
                         n_estimators=n_estimators,
                         max_depth=max_depth,
                         min_child_weight=min_child_weight,
                         gamma=gamma,
                         subsample=subsample,
                         colsample_bytree=colsample_bytree)
    return xgb1  

In [ ]:
#Function to return the list of predictors

In [ ]:
#these are the initial parameters before tuning
def drop_features(l):
    return [x for x in train.columns if x not in l]

### First Prediction : Use of initial parameters and without feature engineering

In [ ]:
predictors = drop_features([target, IDcol])
xgb1=XgbClass()
first_model=modelfit(xgb1, train, predictors)
xgb1.fit(train[predictors],train['Criminal'])

In [ ]:
get_accuracy(first_model,predictors)

In [ ]:
get_feature_importances(first_model)

### Second Prediction : Using intial Parameters and removing features of least importances

In [ ]:
#model after removing the features of least importance

In [ ]:
dropl=['IRWELMOS','MAIIN102','IIPINC3','HLTINNOS','IIHH65_2','TOOLONG']

In [ ]:
dropl_first=dropl+[target,IDcol]

In [ ]:
#these are the initial parameters before tuning
predictors = drop_features(dropl_first)
xgb1 = XgbClass()
second_model=modelfit(xgb1, train, predictors)

In [ ]:
get_accuracy(second_model,predictors)

In [ ]:
get_feature_importances(second_model)

### Third Prediction : Again removing the features of least importance

In [ ]:
dropl1=dropl+['IRMCDCHP','HLCLAST','IIKI17_2','IRFAMPMT','IRFSTAMP','ANYHLTI2','IIFAMSVC']
dropl_second=dropl_first+['IRMCDCHP','HLCLAST','IIKI17_2','IRFAMPMT','IRFSTAMP','ANYHLTI2',
                          'IIFAMSVC']

In [ ]:
predictors=drop_features(dropl_second)

In [ ]:
xgb1=XgbClass()
third_model1=modelfit(xgb1,train,predictors)

In [ ]:
get_accuracy(third_model1,predictors)

## 06 : Tune the Model

In [ ]:
def RunTestAndSaveResults(features,filename,model):
    df1=pd.read_csv('criminal_test.csv')
    for i in features:
        df1.drop(i,axis=1,inplace=True)
    predict=model.predict(df1.drop('PERID',axis=1))
    data=pd.DataFrame(df1['PERID'],columns=['PERID'])
    data['Criminal']=predict
    data.to_csv(filename,index=False)

In [ ]:
#tune max_depth and min_child_weight

In [ ]:
predictors = drop_features(dropl_first)

In [ ]:
param_test1 = {
 'max_depth':list(range(3,10,2)),
 'min_child_weight':list(range(1,6,2))
}
gsearch1 = GridSearchCV(estimator=XgbClass(n_estimators=48),param_grid =param_test1,
                        scoring='roc_auc',n_jobs=-1,iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
#if the best parameters are edge values then we do 
#gridsearchcv by taking one less and one value more than the best parameters

In [ ]:
param_test2 = {
 'max_depth':[6,7,8,9],
 'min_child_weight':[2,3,4,5]
}
gsearch2 = GridSearchCV(estimator=XgbClass(n_estimators=48),param_grid =param_test2,scoring='roc_auc',n_jobs=-1,iid=False, cv=5)
gsearch2.fit(train[predictors],train[target])
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

In [ ]:
xgb1 = XgbClass(max_depth=8,min_child_weight=4)
model=modelfit(xgb1, train, predictors)

In [ ]:
get_accuracy(model,predictors)

In [ ]:
#to tune gamma

In [ ]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,8)]
}
gsearch3=GridSearchCV(estimator=XgbClass(n_estimators=48,max_depth=7,min_child_weight=5),
                      param_grid =param_test3,scoring='roc_auc',n_jobs=-1,iid=False, cv=5)
gsearch3.fit(train[predictors],train[target])
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

In [ ]:
xgb1 = XgbClass(max_depth=8,min_child_weight=4,gamma=0.4)
model=modelfit(xgb1, train, predictors)

In [ ]:
get_accuracy(model,predictors)

In [ ]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4=GridSearchCV(estimator=XgbClass(n_estimators=48,max_depth=7,
                                         min_child_weight=5,gamma=0),
                      param_grid =param_test4,scoring='roc_auc',n_jobs=-1,iid=False, cv=5)
gsearch4.fit(train[predictors],train[target])
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

In [ ]:
xgb1 = XgbClass(max_depth=8,min_child_weight=4,gamma=0.4,subsample=0.8,colsample_bytree=0.6)
model=modelfit(xgb1, train, predictors)

In [ ]:
get_accuracy(model,predictors)

In [ ]:
dropl=['IRWELMOS','MAIIN102','IIPINC3','HLTINNOS','IIHH65_2','TOOLONG']
dropl =dropl+ ['HLCLAST','IIFAMSVC', 'IIKI17_2', 'ANYHLTI2', 'IRFAMPMT', 'IRFSTAMP', 'IRMCDCHP']
RunTestAndSaveResults(dropl,'final_result.csv',third_model1)

## 07 : Predicting on New Data

In [ ]:
#Function stores the result in required csv file and features

In [ ]:
RunTestAndSaveResults([],'result.csv',first_model)

In [ ]:
#This model is giving high accurancy since we applied feature engineering

In [ ]:
dropl

In [ ]:
RunTestAndSaveResults(dropl,'result.csv',second_model)

In [ ]:
#When the features are repeated again and again then overfitting takes place and the accuracy decrease

In [ ]:
RunTestAndSaveResults(dropl1,'result1.csv',third_model1)